In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from sklearn.neighbors import KernelDensity

# ▶ 1. POI 데이터 불러오기 및 좌표계 변환
df_poi = pd.read_csv("result\세종특별자치시_POI_최종.csv")
gdf_poi = gpd.GeoDataFrame(
    df_poi,
    geometry=gpd.points_from_xy(df_poi['x'], df_poi['y']),
    crs="EPSG:4326"
).to_crs(epsg=5181)

# ▶ 2. 어울링 대여소 데이터 불러오기
df_bike = pd.read_csv("data/세종도시교통공사_공공자전거(어울링)_대여소현황_20250106.csv", encoding="euc-kr")

# 위도/경도 컬럼 자동 인식 (일반적으로 lat, lon 또는 위도, 경도)
lon_col = [col for col in df_bike.columns if 'lon' in col.lower() or '경도' in col][0]
lat_col = [col for col in df_bike.columns if 'lat' in col.lower() or '위도' in col][0]

gdf_bike = gpd.GeoDataFrame(
    df_bike,
    geometry=gpd.points_from_xy(df_bike[lon_col], df_bike[lat_col]),
    crs="EPSG:4326"
).to_crs(epsg=5181)

# ▶ 3. KDE 학습 (Epanechnikov 커널, 대역폭 1km)
coords = np.vstack([gdf_poi.geometry.x, gdf_poi.geometry.y]).T
coords = coords[~np.isnan(coords).any(axis=1)]  # NaN 제거
coords = coords[np.all(np.isfinite(coords), axis=1)]  # inf 제거

# KDE 학습
kde = KernelDensity(kernel='epanechnikov', bandwidth=1000).fit(coords)

# ▶ 4. 대여소 위치에 대한 KDE score 계산
bike_coords = np.vstack([gdf_bike.geometry.x, gdf_bike.geometry.y]).T
scores = np.exp(kde.score_samples(bike_coords))  # log density → 밀도

# ▶ 5. 밀도 컬럼 추가 및 저장
# 정규화된 활동지수 컬럼 추가
gdf_bike["kde_density_norm"] = (scores - scores.min()) / (scores.max() - scores.min())
# CSV 저장
gdf_bike.drop(columns="geometry").to_csv("result/어울링_대여소_with_kde.csv", index=False, encoding="utf-8-sig")